# Data & Imports

In [1]:
%reload_ext nb_black
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import plotly
import seaborn as sns

# from statsmodels.graphics.gofplots import qqplot
%matplotlib inline

<IPython.core.display.Javascript object>

In [16]:
# from matplotlib.mlab import PCA as mlabPCA # deprecated
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [2]:
%reload_ext nb_black
postgres_user = "dsbc_student"
postgres_pw = "7*.8G9QH21"
postgres_host = "142.93.121.174"
postgres_port = "5432"
postgres_db = "useducation"
conn_str = f"postgresql://{postgres_user}:{postgres_pw}@{postgres_host}:{postgres_port}/{postgres_db}"
query = """
SELECT *
FROM useducation
"""
df = pd.read_sql_query(query, con=conn_str)
df.head()

,PRIMARY_KEY,STATE,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,...,GRADES_4_G,GRADES_8_G,GRADES_12_G,GRADES_1_8_G,GRADES_9_12_G,GRADES_ALL_G,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE
0,1992_ALABAMA,ALABAMA,1992,NaN,2678885.0,304177.0,1659028.0,715680.0,2653798.0,1481703.0,...,57948.0,58025.0,41167.0,471564.0,196386.0,676174.0,208.327876,252.187522,207.963517,NaN
1,1992_ALASKA,ALASKA,1992,NaN,1049591.0,106780.0,720711.0,222100.0,972488.0,498362.0,...,9748.0,8789.0,6714.0,79117.0,30847.0,112335.0,NaN,NaN,NaN,258.859712
2,1992_ARIZONA,ARIZONA,1992,NaN,3258079.0,297888.0,1369815.0,1590376.0,3401580.0,1435908.0,...,55433.0,49081.0,37410.0,437127.0,175210.0,614881.0,215.253932,265.366278,206.212716,262.169895
3,1992_ARKANSAS,ARKANSAS,1992,NaN,1711959.0,178571.0,958785.0,574603.0,1743022.0,964323.0,...,34632.0,36011.0,27651.0,281338.0,123113.0,405259.0,210.206028,256.312090,208.634458,264.619665
4,1992_CALIFORNIA,CALIFORNIA,1992,NaN,26260025.0,2072470.0,16546514.0,7641041.0,27138832.0,14358922.0,...,418418.0,363296.0,270675.0,3286034.0,1372011.0,4717112.0,208.398961,260.892247,196.764414,NaN


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [5]:
fill_list = [
    "ENROLL",
    "TOTAL_REVENUE",
    "FEDERAL_REVENUE",
    "STATE_REVENUE",
    "LOCAL_REVENUE",
    "TOTAL_EXPENDITURE",
    "INSTRUCTION_EXPENDITURE",
    "SUPPORT_SERVICES_EXPENDITURE",
    "OTHER_EXPENDITURE",
    "CAPITAL_OUTLAY_EXPENDITURE",
    "GRADES_PK_G",
    "GRADES_KG_G",
    "GRADES_4_G",
    "GRADES_8_G",
    "GRADES_12_G",
    "GRADES_1_8_G",
    "GRADES_9_12_G",
    "GRADES_ALL_G",
]

states = df["STATE"].unique()

for state in states:
    df.loc[df["STATE"] == state, fill_list] = df.loc[
        df["STATE"] == state, fill_list
    ].interpolate()

# we drop the null values after interpolation
df.dropna(inplace=True)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### 1. Compute the average score for each row for all students weighted properly by grade. In other words, each row will have its own average. Notice that the number of students in the fourth grade isn't the same as the number of students in the eighth grade. So, you should appropriately weigh the scores.

In [8]:
# var for the total students in 4th and 8th grades
total_students = df.GRADES_4_G + df.GRADES_8_G

df["overall_avg"] = (
    (
        (df.AVG_MATH_4_SCORE * (df.GRADES_4_G / total_students))
        + (df.AVG_MATH_8_SCORE * (df.GRADES_8_G / total_students))
    )
    + (
        (df.AVG_READING_4_SCORE * (df.GRADES_4_G / total_students))
        + (df.AVG_READING_8_SCORE * (df.GRADES_8_G / total_students))
    )
) / 2

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

---

### 2. What are the correlations between this newly created score variable and the expenditure types? Which one of the expenditure types is more correlated than the others?

In [10]:
df.head()

,PRIMARY_KEY,STATE,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,...,GRADES_8_G,GRADES_12_G,GRADES_1_8_G,GRADES_9_12_G,GRADES_ALL_G,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE,overall_avg
209,1996_ALABAMA,ALABAMA,1996,735912.0,3365113.0,334858.0,2138717.0,891538.0,3572283.0,1987018.0,...,58305.0,42510.0,469370.0,207980.0,686983.0,211.646974,256.594863,210.920767,260.276441,235.106342
211,1996_ARIZONA,ARIZONA,1996,764681.0,4143304.0,378917.0,1798363.0,1966024.0,4391555.0,1860007.0,...,60514.0,42041.0,511925.0,210289.0,726709.0,217.575940,267.874834,206.529788,263.283638,238.321404
212,1996_ARKANSAS,ARKANSAS,1996,452907.0,2183384.0,192152.0,1296247.0,694985.0,2297381.0,1264275.0,...,37064.0,27613.0,283883.0,131997.0,417860.0,215.846436,261.652745,208.837716,271.834590,240.773675
213,1996_CALIFORNIA,CALIFORNIA,1996,5460484.0,31282981.0,2603882.0,17061474.0,11617625.0,31561692.0,16880730.0,...,406456.0,298669.0,3516572.0,1525618.0,5115423.0,209.129083,262.772478,201.912591,269.197451,234.719834
214,1996_COLORADO,COLORADO,1996,655679.0,3891203.0,194998.0,1664676.0,2031529.0,4075165.0,2052264.0,...,52269.0,37179.0,423510.0,185700.0,621730.0,225.805858,275.608214,221.653243,268.168154,247.750273


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [13]:
expenditures = df[
    [
        "INSTRUCTION_EXPENDITURE",
        "SUPPORT_SERVICES_EXPENDITURE",
        "OTHER_EXPENDITURE",
        "CAPITAL_OUTLAY_EXPENDITURE",
        "overall_avg",
    ]
]
expenditures.head()

,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE,overall_avg
209,1987018.0,935115.0,322883.0,286699.0,235.106342
211,1860007.0,1193260.0,220650.0,877190.0,238.321404
212,1264275.0,636106.0,127007.0,203555.0,240.773675
213,16880730.0,9222569.0,2362407.0,2580639.0,234.719834
214,2052264.0,1127563.0,143767.0,588209.0,247.750273


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [14]:
expenditures.corr()

,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE,overall_avg
INSTRUCTION_EXPENDITURE,1.000000,0.974399,0.902173,0.884798,0.087386
SUPPORT_SERVICES_EXPENDITURE,0.974399,1.000000,0.952282,0.911245,0.079002
OTHER_EXPENDITURE,0.902173,0.952282,1.000000,0.918188,0.010959
CAPITAL_OUTLAY_EXPENDITURE,0.884798,0.911245,0.918188,1.000000,-0.005389
overall_avg,0.087386,0.079002,0.010959,-0.005389,1.000000


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

> Instruction expenditure is most correlated with the overall score avg var
---

### 3. Now, apply PCA to the four expenditure types. How much of the total variance is explained by the first component?

In [33]:
# scaling the data before PCA
# df for just expenditure types
X = expenditures.drop(columns="overall_avg")

# instance of standard scaler
# "learn" from the data with fit_transform()
X = StandardScaler().fit_transform(X)

# create PCA object with n=1
sklearn_pca = PCA(n_components=1)

df["expenditures_pca"] = sklearn_pca.fit_transform(X)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [35]:
print(
    "The percentage of total variance in the dataset explained by each",
    "component from Sklearn PCA.\n",
    sklearn_pca.explained_variance_ratio_ * 100,
)

The percentage of total variance in the dataset explained by each component from Sklearn PCA.
 [94.30174978]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

> 94.3% of the variance is explained by the first component
---

### 4. What is the correlation between the overall score variable and the first principal component?

In [39]:
df[
    [
        "overall_avg",
        "expenditures_pca",
        "TOTAL_EXPENDITURE",
        "INSTRUCTION_EXPENDITURE",
        "SUPPORT_SERVICES_EXPENDITURE",
        "OTHER_EXPENDITURE",
        "CAPITAL_OUTLAY_EXPENDITURE",
    ]
].corr()

,overall_avg,expenditures_pca,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE
overall_avg,1.000000,0.044596,0.074155,0.087386,0.079002,0.010959,-0.005389
expenditures_pca,0.044596,1.000000,0.992054,0.968606,0.988345,0.971343,0.955792
TOTAL_EXPENDITURE,0.074155,0.992054,1.000000,0.990255,0.992988,0.943702,0.925784
INSTRUCTION_EXPENDITURE,0.087386,0.968606,0.990255,1.000000,0.974399,0.902173,0.884798
SUPPORT_SERVICES_EXPENDITURE,0.079002,0.988345,0.992988,0.974399,1.000000,0.952282,0.911245
OTHER_EXPENDITURE,0.010959,0.971343,0.943702,0.902173,0.952282,1.000000,0.918188
CAPITAL_OUTLAY_EXPENDITURE,-0.005389,0.955792,0.925784,0.884798,0.911245,0.918188,1.000000


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

> Correlation between overall_avg and expenditures_pca = 0.044596
---

### 5. If you were to choose the best variables for your model, would you prefer using the first principal component instead of the expenditure variables? Why?

> I would rather use the expenditures vars because they have a higher correlation than the pca var
---